# Tools with MCP ⏰

The Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `langchain-mcp-adapters`.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****lfgA
LANGSMITH_API_KEY=****ec40
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****itas


In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# Connect to the mcp-time server for timezone-aware operations
# This Go-based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic, and time comparison
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "node",
            "args": ["/home/david/uc_chile/proyecto_aplicado_1/PubChem-MCP-Server/build/index.js"],
        }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

Loaded 30 MCP tools: ['search_compounds', 'get_compound_info', 'search_by_smiles', 'search_by_inchi', 'search_by_cas_number', 'get_compound_synonyms', 'search_similar_compounds', 'substructure_search', 'superstructure_search', 'get_3d_conformers', 'analyze_stereochemistry', 'get_compound_properties', 'calculate_descriptors', 'predict_admet_properties', 'assess_drug_likeness', 'analyze_molecular_complexity', 'get_pharmacophore_features', 'search_bioassays', 'get_assay_info', 'get_compound_bioactivities', 'search_by_target', 'compare_activity_profiles', 'get_safety_data', 'get_toxicity_info', 'assess_environmental_fate', 'get_regulatory_info', 'get_external_references', 'search_patents', 'get_literature_references', 'batch_compound_lookup']


Create an agent with the MCP-provided time tools.

In [4]:
from langchain.agents import create_agent

agent_with_mcp = create_agent(
    model="openai:gpt-5",
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

Ask about the current time in San Francisco.

In [5]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "Cual es el compuesto de la cafeina?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Cual es el compuesto de la cafeina?
================================== Ai Message ==================================

La cafeína es un alcaloide del grupo de las metilxantinas. Datos clave:
- Fórmula molecular: C8H10N4O2
- Nombre IUPAC: 1,3,7-trimetilxantina (también: 1,3,7-trimetilpurina-2,6-diona)
- Masa molar: ≈194.19 g/mol
- Estructura: núcleo de xantina (purina) con tres grupos metilo en N-1, N-3 y N-7
- CAS: 58-08-2; PubChem CID: 2519

Si buscas otra cosa (por ejemplo, su estructura 2D/3D o propiedades), dime y te la doy.
